In [7]:
import pandas as pd
import random
import os
import random
from faker import Faker
fake = Faker()

kota = [
    "Jakarta",
    "Surabaya",
    "Bandung",
    "Medan",
    "Semarang",
    "Palembang",
    "Makassar",
    "Depok",
    "Tangerang",
    "Bekasi",
    "Bogor",
    "Denpasar",
    "Malang",
    "Yogyakarta",
    "Pekanbaru",
    "Pontianak",
    "Balikpapan",
    "Banjarmasin",
    "Samarinda",
    "Manado",
    "Padang",
    "Batam",
    "Cirebon",
    "Mataram",
    "Kupang",
    "Jayapura",
    "Ambon",
    "Banda Aceh",
    "Tanjung Pinang",
    "Palangkaraya",
    "Tarakan",
    "Palu",
    "Kendari",
    "Gorontalo",
    "Bengkulu",
    "Serang",
    "Jambi",
    "Cilegon",
    "Tasikmalaya",
    "Probolinggo",
    "Kediri",
    "Madiun",
    "Purwokerto",
    "Solo",
    "Magelang",
    "Sukabumi",
    "Tegal",
    "Cimahi",
    "Salatiga",
    "Pematangsiantar",
    "Lhokseumawe"
]

manager = [
    "M0001",
    "M0002",
    "M0003",
    "M0004",
    "M0005",
    "M0006"
]



In [8]:
# Define the number of rows and the column structure for the synthetic dataset
num_rows = 1000  # Change this to the desired number of rows
columns = [
    {"name": "id_employee", "type": "custom_id"},
    {"name": "first_name", "type": "first_name"},
    {"name": "last_name", "type": "last_name"},
    {"name": "gender", "type": "gender"},
    {"name": "phone", "type": "phone"},
    {"name": "address", "type": "address"},
    {"name": "id_manager", "type": "id_manager"},
    {"name": "id_shift", "type" : "shift_id"}
]

# Generate synthetic data
synthetic_data = pd.DataFrame()

# Generate supporting columns first
first_names = [fake.first_name() for _ in range(num_rows)]
last_names = [fake.last_name() for _ in range(num_rows)]
phones = [f"{random.randint(1000000000, 9999999999)}" for _ in range(num_rows)]

for column in columns:
    if column["type"] == "custom_id":
        synthetic_data[column["name"]] = [f"E{str(i).zfill(4)}" for i in range(1, num_rows + 1)]
    elif column["type"] == "first_name":
        synthetic_data[column["name"]] = first_names
    elif column["type"] == "last_name":
        synthetic_data[column["name"]] = last_names
    elif column["type"] == "gender":
        synthetic_data[column["name"]] = [random.choice(["M", "F"]) for _ in range(num_rows)]
    elif column["type"] == "phone":
        synthetic_data[column["name"]] = phones
    elif column["type"] == "address":
        synthetic_data[column["name"]] = [random.choice(kota) for _ in range(num_rows)]
    elif column["type"] == "id_manager":
        synthetic_data[column["name"]] = [random.choice(manager) for _ in range(num_rows)]
    elif column["type"] == "shift_id":
        synthetic_data[column["name"]] = [f"S{random.randint(1, 3)}" for _ in range(num_rows)]
    else:
        synthetic_data[column["name"]] = [f"Data_{i}" for i in range(1, num_rows + 1)]

output_path = "Contoh2.xlsx"
synthetic_data.to_excel(output_path, index=False)

print(f"Synthetic dataset saved to {output_path}")

Synthetic dataset saved to Contoh2.xlsx


In [3]:
import pandas as pd
import mysql.connector

# Define your MariaDB connection
server = 'Evander'  # Your server name (hostname)
username = 'root'    # Your MariaDB username (root)
password = ''        # No password as you mentioned
database = 'test2'   # The database you want to insert data into
table_name = 'employee'  # The table to insert data into

try:
    # Establish connection
    conn = mysql.connector.connect(
        host=server,
        user=username,
        password=password,
        database=database
    )
    cursor = conn.cursor()
    print("Connection to the database was successful.")
except mysql.connector.Error as err:
    print(f"Error: {err}")
    exit()

# Load the dataset
input_file = "Registered_Employee_Table.xlsx"
synthetic_data = pd.read_excel(input_file)

# Debugging: Display column names and data
print("Columns in Excel file:", synthetic_data.columns)
print("First few rows:\n", synthetic_data.head())

# Ensure the Excel columns match the database table
synthetic_data.rename(columns={
    'first_name': 'first_name_e',
    'last_name': 'last_name_e',
    'gender': 'gender_e',
    'phone': 'phone_num_e',
    'address': 'alamat_e'
}, inplace=True)

# Insert data into MariaDB
try:
    for index, row in synthetic_data.iterrows():
        cursor.execute(f"""
            INSERT INTO {table_name} (id_employee, first_name_e, last_name_e, gender_e, phone_num_e, alamat_e, id_manager, id_shift)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        """, 
        (
            row['id_employee'], 
            row['first_name_e'], 
            row['last_name_e'], 
            row['gender_e'], 
            row['phone_num_e'], 
            row['alamat_e'], 
            row['id_manager'], 
            row['id_shift']
        ))

    conn.commit()
    print("Data successfully inserted into MariaDB!")
except mysql.connector.Error as err:
    print(f"Error inserting data: {err}")
except KeyError as e:
    print(f"Missing column in Excel file: {e}")
finally:
    # Close the database connection
    cursor.close()
    conn.close()
    print("Database connection closed.")


Connection to the database was successful.
Columns in Excel file: Index(['id_employee', 'first_name', 'last_name', 'gender', 'phone', 'address',
       'id_manager', 'id_shift'],
      dtype='object')
First few rows:
   id_employee first_name last_name gender       phone    address id_manager  \
0       E0001     Jeremy    Mosley      M  4390184384  Gorontalo      M0005   
1       E0002      Isaac    Gordon      M  6950749781   Bengkulu      M0002   
2       E0003     Pamela      Paul      M  3738563708   Surabaya      M0004   
3       E0004    Gregory      Ford      F  7128408422   Bengkulu      M0003   
4       E0005      Brent     Greer      M  3444087799  Gorontalo      M0005   

  id_shift  
0       S1  
1       S3  
2       S2  
3       S1  
4       S2  
Data successfully inserted into MariaDB!
Database connection closed.
